In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
import re
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
shot = 120
picL = 18333
picR = 18158
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

In [2]:
tL = pd.read_csv('./Data/Shot{}/trackL3_frame_inv.csv'.format(shot))
tL.head()
#tL['particle'].unique()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,10,NaN,3.0,NaN,NaN,NaN,0.014218,0.000708,10,143.608874,44.0,425.0,37.287918,0.860436,303.954386,172.007018,71.287194
1,11,NaN,3.0,NaN,NaN,NaN,0.420543,0.001101,11,295.025891,51.0,1050.0,74.717122,0.864143,281.981187,111.004342,118.541388
2,12,1.0,3.0,5.0,8.0,2.0,0.106667,0.002879,12,414.409554,44.0,1270.0,97.846700,0.874007,304.034259,170.009259,71.287194
3,12,NaN,3.0,NaN,NaN,NaN,0.408875,0.002868,12,313.876866,51.0,1275.0,71.754247,0.878290,280.993888,110.014670,118.541388
4,13,NaN,3.0,NaN,NaN,NaN,0.176101,0.002907,13,336.087171,44.0,1275.0,74.136876,0.882843,304.083333,169.007843,71.287194


In [3]:
tR = pd.read_csv('./Data/Shot{}/trackR3_frame_inv.csv'.format(shot))
tR.head()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,9,NaN,3.0,NaN,NaN,NaN,0.335821,0.000000,9,35.045209,21.0,69.0,9.661112,0.851072,101.097297,129.151351,263.281486
1,9,NaN,3.0,NaN,NaN,NaN,0.587719,0.000000,9,29.741069,24.0,83.0,8.145643,0.852124,135.082803,108.019108,68.873961
2,9,NaN,3.0,NaN,NaN,NaN,0.242424,0.000000,9,16.670153,32.0,19.0,4.167538,0.866025,197.022727,118.977273,132.527775
3,9,NaN,3.0,NaN,NaN,NaN,0.205479,0.000000,9,19.132790,36.0,36.0,5.304140,0.850160,204.000000,137.029703,153.799773
4,10,NaN,3.0,NaN,NaN,NaN,0.350254,0.001011,10,129.166983,21.0,280.0,33.864003,0.858969,101.880150,129.921348,263.281486


In [4]:
## Print out particle track list in the left camera
listL = tL['particle'].unique()
listL = listL.astype(int)
listR = tR['particle'].unique()
listR = listR.astype(int)

In [ ]:
##Try to pair particle track between two cameras.
##Loop particle track in the left camera
#Pair = []
for ilistL in listL:
#for ilistL in range(0,1):
    Pair = []
    ipL0 = int(ilistL)
    TrackL = tL[tL['particle']==ipL0]
    lenL = len(TrackL)
    if ipL0>=1188 and lenL>400:
        print(ipL0,lenL)    
        xmaxL = np.max(TrackL['x'])
        xminL = np.min(TrackL['x'])
        ymaxL = np.max(TrackL['y'])
        yminL = np.min(TrackL['y'])
        for ilistR in listR:
            MassDiffSum = 0
            FFSum = 0
            FrameCount = 0
            MassDiffAve = 0
            FFAve = 0
            FFSmallCount = 0
            ipR0 = ilistR
            TrackR = tR[tR['particle']==ipR0] 
            lenR = len(TrackR)
            xmaxR = np.max(TrackR['x'])
            xminR = np.min(TrackR['x'])
            ymaxR = np.max(TrackR['y'])
            yminR = np.min(TrackR['y'])
            for iframeL in TrackL['frame'].unique():
                for iframeR in TrackR['frame'].unique():
                    if(iframeL == iframeR):
                        FrameCount = FrameCount+1
                        TrackL1=TrackL[TrackL['frame']==iframeL]
                        TrackR1=TrackR[TrackR['frame']==iframeR]
                        imassL = TrackL1['mass'].iloc[0]
                        imassR = TrackR1['mass'].iloc[0]
                        MassDiff = (imassL-imassR)*(imassL-imassR)
                        MassDiffSum = MassDiffSum+MassDiff
                        ixL = TrackL1['x'].iloc[0]
                        iyL = TrackL1['y'].iloc[0]  
                        ixR = TrackR1['x'].iloc[0]
                        iyR = TrackR1['y'].iloc[0]                         
                        ipL = [ixL,iyL,1]                       
                        ipR = [ixR,iyR,1]
                        LineL = np.dot(ipR,MatrixL) 
                        LineR = np.dot(ipL,MatrixR) 
                        #LineLy = (-LineL[2]-LineL[0]*Linex)/LineL[1]
                        #LineRy = (-LineR[2]-LineR[0]*Linex)/LineR[1]  
                        dlr = math.fabs(np.dot(ipL,LineL))/math.sqrt(LineL[0]*LineL[0]+LineL[1]*LineL[1])
                        drl = math.fabs(np.dot(ipR,LineR))/math.sqrt(LineR[0]*LineR[0]+LineR[1]*LineR[1])
                        FF = dlr + drl
                        if(FF<5):
                            FFSmallCount = FFSmallCount+1
                        FFSum = FFSum + FF
            if(FrameCount>0):
                MassDiffAve = math.sqrt(MassDiffSum/FrameCount)
                FFAve = FFSum/FrameCount
                templist = [ipL0,ipR0,lenL,lenR,xminL,xmaxL,yminL,ymaxL,xminR,xmaxR,yminR,ymaxR,FrameCount,MassDiffAve,FFAve,FFSmallCount]
                Pair.append(templist) 
                P1 = pd.DataFrame(Pair)
                P1.columns = ['PIDL','PIDR','LenL','LenR','Xmin L','Xmax L','Ymin L','Ymax L','Xmin R','Xmax R','Ymin R','Ymax R','Frame Count','Mass Diff','FF Ave','FF Small Count']
                P1.to_csv("./Data/Shot{}/TrackPair_PID_{}.csv".format(shot,ipL0))